# 🎭 Fine-tuning LLM на мероприятиях Алматы с QLoRA + ORPO

## Обзор проекта

В этом ноутбуке мы обучаем языковую модель **Qwen2.5-3B** на датасете по мероприятиям Алматы (sxodim.com), используя:
- **QLoRA** — для эффективного fine-tuning с 4-bit квантизацией
- **ORPO** — для дружелюбного, человечного стиля ответов

### Пайплайн обучения:
```
База (Qwen2.5-3B) → SFT (знание фактов) → ORPO (дружелюбный стиль)
```

### Что такое QLoRA + ORPO?

| Метод | Цель | Результат |
|-------|------|-----------|
| **QLoRA** | Эффективное обучение | ~6GB памяти вместо 48GB |
| **SFT** | Научить фактам | Модель знает о мероприятиях |
| **ORPO** | Научить стилю | Отвечает дружелюбно и живо |

### Структура ноутбука:
1. **Установка зависимостей**
2. **Загрузка модели с 4-bit квантизацией**
3. **Настройка QLoRA адаптеров**
4. **Подготовка датасета**
5. **SFT обучение** (факты о мероприятиях)
6. **ORPO обучение** (дружелюбный стиль)
7. **Оценка модели**
8. **Интерактивное тестирование**
9. **Экспорт модели**

## 1. Установка зависимостей

Используем **Unsloth** — библиотеку для ускоренного fine-tuning LLM.

Unsloth обеспечивает:
- 2x ускорение обучения
- 60% меньше памяти
- Оптимизированные CUDA kernels

In [ ]:
%%capture
# Установка через uv (быстрый пакетный менеджер)
!pip install --upgrade -qqq uv

# 1. Базовые зависимости для deep learning
!uv pip install -qqq \
  "torch>=2.8.0" \
  "triton>=3.4.0" \
  torchvision \
  bitsandbytes==0.48.0 \
  transformers==4.56.2

# 2. Unsloth (ВАЖНО: устанавливаем с GitHub, не из PyPI!)
!uv pip install -qqq \
  "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
  "unsloth[base] @ git+https://github.com/unslothai/unsloth"

# 3. TRL для SFT обучения (без зависимостей, чтобы избежать конфликтов)
!uv pip install -qqq --no-deps trl==0.22.2

# 4. Метрики для оценки
!uv pip install -qqq evaluate bert_score sacrebleu rouge_score

In [ ]:
# Импорты
import json
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from tqdm import tqdm

# Проверка GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Загрузка модели с 4-bit квантизацией (QLoRA)

### Выбор модели: Qwen2.5-3B

**Qwen2.5-3B** — современная модель от Alibaba:
- 3 миллиарда параметров
- Отличная поддержка русского и казахского языков
- Контекст до 32K токенов
- Оптимизирована для instruction-following

### Параметры квантизации:
- `load_in_4bit=True` — загружаем веса в 4-bit формате (NF4)
- `bnb_4bit_compute_dtype=float16` — вычисления в FP16
- `bnb_4bit_quant_type="nf4"` — NormalFloat4 квантизация
- `bnb_4bit_use_double_quant=True` — двойная квантизация (экономит ещё ~0.4 бит/параметр)

In [ ]:
# Конфигурация модели
MODEL_NAME = "unsloth/Qwen2.5-3B-bnb-4bit"  # Предквантизированная 4-bit версия
MAX_SEQ_LENGTH = 2048                        # Максимальная длина последовательности

# Загрузка модели и токенизатора
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,              # QLoRA: 4-bit квантизация
    dtype=None,                     # Auto-detect (float16 для GPU)
)

print(f"\n✅ Модель загружена: {MODEL_NAME}")
print(f"📊 Тип квантизации: 4-bit NF4 (QLoRA)")

## 3. Настройка QLoRA адаптеров

### Параметры LoRA:

| Параметр | Значение | Описание |
|----------|----------|----------|
| `r` | 32 | Ранг матриц A и B. Выше = больше параметров, лучше качество |
| `lora_alpha` | 64 | Коэффициент масштабирования. Обычно `alpha = 2 * r` |
| `lora_dropout` | 0.05 | Dropout для регуляризации |
| `target_modules` | [q, k, v, o, gate, up, down] | Слои для адаптации |

### Целевые модули:
- **Attention**: `q_proj`, `k_proj`, `v_proj`, `o_proj`
- **MLP/FFN**: `gate_proj`, `up_proj`, `down_proj`

Адаптируем все ключевые слои для максимального качества.

In [ ]:
# Настройка QLoRA адаптеров
model = FastLanguageModel.get_peft_model(
    model,
    r=32,                    # Ранг LoRA (выше для лучшего качества)
    lora_alpha=64,           # Alpha = 2 * r (стандартная практика)
    lora_dropout=0.05,       # Небольшой dropout для регуляризации
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
        "gate_proj", "up_proj", "down_proj"       # MLP layers
    ],
    bias="none",             # Не обучаем bias (экономия памяти)
    use_gradient_checkpointing="unsloth",  # Экономия памяти при backprop
    random_state=42,
)

# Статистика параметров
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
frozen = total - trainable

print(f"\n📊 Статистика параметров:")
print(f"   Всего параметров:    {total:>15,}")
print(f"   Замороженных:        {frozen:>15,} ({100*frozen/total:.2f}%)")
print(f"   Обучаемых (QLoRA):   {trainable:>15,} ({100*trainable/total:.2f}%)")
print(f"\n✅ QLoRA адаптеры настроены!")

## 4. Подготовка датасета

Загружаем SFT датасет по мероприятиям Алматы (sxodim.com).

### Формат данных:
```json
{
  "messages": [
    {"role": "system", "content": "Ты — дружелюбный помощник по мероприятиям в Алматы..."},
    {"role": "user", "content": "Куда сходить на выходных?"},
    {"role": "assistant", "content": "Рекомендую посетить концерт..."}
  ]
}
```

### Формат чата Qwen2.5 (ChatML):
```
<|im_start|>system
Ты — помощник по мероприятиям в Алматы.<|im_end|>
<|im_start|>user
Вопрос?<|im_end|>
<|im_start|>assistant
Ответ.<|im_end|>
```

In [ ]:
# Загружаем датасет
DATASET_PATH = "/content/sft_dataset_chat.json"  # Путь к датасету мероприятий Алматы

with open(DATASET_PATH, "r", encoding="utf-8") as f:
    sft_data = json.load(f)

print(f"📚 Загружено примеров: {len(sft_data)}")

# Показать пример
print("\n📝 Пример из датасета:")
example = sft_data[50]
for msg in example["messages"]:
    role = msg["role"].upper()
    content = msg["content"][:100] + "..." if len(msg["content"]) > 100 else msg["content"]
    print(f"   [{role}]: {content}")

In [ ]:
# Настройка chat template для Qwen2.5 (ChatML формат)
if tokenizer.chat_template is None:
    tokenizer.chat_template = (
        "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
        "{{ '<|im_start|>system\n' + message['content'] + '<|im_end|>\n' }}"
        "{% elif message['role'] == 'user' %}"
        "{{ '<|im_start|>user\n' + message['content'] + '<|im_end|>\n' }}"
        "{% elif message['role'] == 'assistant' %}"
        "{{ '<|im_start|>assistant\n' + message['content'] + '<|im_end|>\n' }}"
        "{% endif %}"
        "{% endfor %}"
        "{% if add_generation_prompt %}"
        "{{ '<|im_start|>assistant\n' }}"
        "{% endif %}"
    )
    print("✅ Chat template установлен")

# Функция форматирования для обучения
def format_for_training(example):
    """Преобразует messages в текстовый формат для SFT."""
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

# Создаём HuggingFace Dataset
dataset = Dataset.from_list(sft_data)
dataset = dataset.map(format_for_training, remove_columns=["metadata"] if "metadata" in dataset.column_names else [])

print(f"\n✅ Датасет подготовлен: {dataset}")

# Показать отформатированный пример
print("\n📝 Пример после форматирования:")
print(dataset[50]["text"][:500] + "...")

## 5. Обучение (SFT с QLoRA)

### Гиперпараметры обучения:

| Параметр | Значение | Описание |
|----------|----------|----------|
| `per_device_train_batch_size` | 2 | Размер батча на GPU |
| `gradient_accumulation_steps` | 4 | Эффективный batch = 2 * 4 = 8 |
| `learning_rate` | 2e-4 | Стандартный LR для LoRA |
| `num_train_epochs` | 3 | Количество эпох |
| `warmup_ratio` | 0.1 | 10% шагов на warmup |
| `lr_scheduler_type` | cosine | Косинусное затухание LR |
| `fp16` | True | Mixed precision training |

### Формула эффективного batch size:
```
effective_batch = per_device_batch * gradient_accumulation * num_gpus
                = 2 * 4 * 1 = 8
```

In [ ]:
# Конфигурация обучения
training_args = TrainingArguments(
    output_dir="./shodim_almaty_qlora_output",
    
    # Batch size
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # Effective batch = 8
    
    # Обучение
    num_train_epochs=3,             # 3 эпохи для полного обучения
    learning_rate=2e-4,             # Стандартный LR для QLoRA
    warmup_ratio=0.1,               # 10% warmup
    lr_scheduler_type="cosine",     # Косинусное затухание
    
    # Оптимизация
    optim="adamw_8bit",             # 8-bit AdamW (экономия памяти)
    weight_decay=0.01,              # L2 регуляризация
    max_grad_norm=1.0,              # Gradient clipping
    
    # Precision
    fp16=True,                      # Mixed precision
    bf16=False,                     # Используем FP16 (более совместимо)
    
    # Логирование
    logging_steps=10,
    logging_first_step=True,
    
    # Сохранение
    save_strategy="epoch",
    save_total_limit=2,             # Храним только 2 последних checkpoint
    
    # Прочее
    seed=42,
    report_to="none",               # Отключаем W&B/TensorBoard
)

# Создаём SFT Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=True,                   # Упаковка коротких примеров (эффективнее)
    args=training_args,
)

# Статистика
total_steps = len(trainer.get_train_dataloader()) * training_args.num_train_epochs
print(f"\n📊 Конфигурация обучения:")
print(f"   Эпох: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Примерно шагов: ~{total_steps}")

In [ ]:
# 🚀 Запуск обучения
print("🚀 Начинаем обучение...\n")

train_result = trainer.train()

# Результаты
print("\n" + "="*50)
print("✅ Обучение завершено!")
print("="*50)
print(f"   Train Loss: {train_result.training_loss:.4f}")
print(f"   Train Runtime: {train_result.metrics['train_runtime']:.1f}s")
print(f"   Samples/sec: {train_result.metrics['train_samples_per_second']:.2f}")

## 5.1 ORPO: Preference Optimization для дружелюбных ответов

### Что такое ORPO?

**ORPO (Odds Ratio Preference Optimization)** — метод, который учит модель предпочитать один стиль ответов другому:

| Аспект | SFT | ORPO |
|--------|-----|------|
| Данные | Только правильные ответы | Пары: chosen (хороший) vs rejected (плохой) |
| Цель | Имитировать ответы | Предпочитать chosen над rejected |
| Результат | Знает факты | Знает факты + правильный тон |

### Зачем нужен ORPO после SFT?

После SFT модель знает **что** отвечать. ORPO учит **как** отвечать:
- **Chosen**: Дружелюбный, живой, человечный стиль
- **Rejected**: Сухой, формальный, роботизированный стиль

### Формат данных ORPO:
```json
{
  "prompt": [{"role": "user", "content": "Куда сходить?"}],
  "chosen": [{"role": "assistant", "content": "О, отличный вопрос! Очень советую..."}],
  "rejected": [{"role": "assistant", "content": "Информируем вас о наличии мероприятий..."}]
}
```

In [ ]:
# Загружаем ORPO датасет с preference парами
ORPO_DATASET_PATH = "/content/orpo_dataset.json"

with open(ORPO_DATASET_PATH, "r", encoding="utf-8") as f:
    orpo_data = json.load(f)

print(f"📚 Загружено preference пар: {len(orpo_data)}")

# Показать пример
print("\n📝 Пример preference пары:")
ex = orpo_data[0]
print(f"   [PROMPT]: {ex['prompt'][1]['content'][:80]}...")
print(f"   [CHOSEN]: {ex['chosen'][0]['content'][:80]}...")
print(f"   [REJECTED]: {ex['rejected'][0]['content'][:80]}...")

In [ ]:
# Подготовка датасета для ORPO
from trl import ORPOTrainer, ORPOConfig

def format_orpo_example(example):
    """Форматируем пример для ORPO тренера."""
    # Prompt (system + user)
    prompt_text = tokenizer.apply_chat_template(
        example["prompt"],
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Chosen response
    chosen_text = tokenizer.apply_chat_template(
        example["prompt"] + example["chosen"],
        tokenize=False,
        add_generation_prompt=False
    )
    
    # Rejected response  
    rejected_text = tokenizer.apply_chat_template(
        example["prompt"] + example["rejected"],
        tokenize=False,
        add_generation_prompt=False
    )
    
    return {
        "prompt": prompt_text,
        "chosen": chosen_text,
        "rejected": rejected_text
    }

# Создаём датасет
orpo_dataset = Dataset.from_list(orpo_data)
orpo_dataset = orpo_dataset.map(format_orpo_example, remove_columns=orpo_dataset.column_names)

print(f"✅ ORPO датасет подготовлен: {orpo_dataset}")

In [ ]:
# Конфигурация ORPO
orpo_config = ORPOConfig(
    output_dir="./shodim_almaty_orpo_output",
    
    # Batch size
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    
    # Обучение
    num_train_epochs=2,              # Меньше эпох для preference tuning
    learning_rate=5e-5,              # Меньший LR для fine-tuning
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    
    # ORPO специфичные параметры
    beta=0.1,                        # Коэффициент для odds ratio loss
    max_length=1024,                 # Максимальная длина
    max_prompt_length=512,           # Максимальная длина промпта
    
    # Оптимизация
    optim="adamw_8bit",
    weight_decay=0.01,
    max_grad_norm=1.0,
    
    # Precision
    fp16=True,
    bf16=False,
    
    # Логирование
    logging_steps=10,
    
    # Сохранение
    save_strategy="epoch",
    save_total_limit=2,
    
    seed=42,
    report_to="none",
)

# Возвращаем модель в режим обучения
FastLanguageModel.for_training(model)

# Создаём ORPO Trainer
orpo_trainer = ORPOTrainer(
    model=model,
    args=orpo_config,
    train_dataset=orpo_dataset,
    tokenizer=tokenizer,
)

print(f"\n📊 ORPO конфигурация:")
print(f"   Beta (odds ratio): {orpo_config.beta}")
print(f"   Learning rate: {orpo_config.learning_rate}")
print(f"   Эпох: {orpo_config.num_train_epochs}")

In [ ]:
# 🚀 Запуск ORPO обучения
print("🚀 Начинаем ORPO обучение (preference optimization)...\n")

orpo_result = orpo_trainer.train()

# Результаты
print("\n" + "="*50)
print("✅ ORPO обучение завершено!")
print("="*50)
print(f"   Train Loss: {orpo_result.training_loss:.4f}")
print(f"   Train Runtime: {orpo_result.metrics['train_runtime']:.1f}s")

# Сохраняем финальную модель
FINAL_OUTPUT_DIR = "shodim_almaty_orpo_final"
model.save_pretrained(FINAL_OUTPUT_DIR)
tokenizer.save_pretrained(FINAL_OUTPUT_DIR)
print(f"\n✅ Финальная модель сохранена в: {FINAL_OUTPUT_DIR}/")

## 6. Сохранение модели

Сохраняем обученные QLoRA адаптеры. Они занимают всего ~100-200 MB.

In [ ]:
# Сохраняем QLoRA адаптеры
OUTPUT_DIR = "shodim_almaty_qlora"

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Модель сохранена в: {OUTPUT_DIR}/")

# Показать размер
import os
total_size = sum(os.path.getsize(os.path.join(OUTPUT_DIR, f)) for f in os.listdir(OUTPUT_DIR))
print(f"📦 Размер адаптеров: {total_size / 1e6:.1f} MB")

## 7. Оценка модели

### Метрики:

1. **BLEU** (Bilingual Evaluation Understudy)
   - Измеряет совпадение n-грамм между generated и reference
   - Диапазон: 0-100, выше = лучше

2. **ROUGE** (Recall-Oriented Understudy for Gisting Evaluation)
   - ROUGE-1: Совпадение униграмм
   - ROUGE-2: Совпадение биграмм
   - ROUGE-L: Longest Common Subsequence
   - Диапазон: 0-1, выше = лучше

3. **BERTScore**
   - Семантическое сходство через BERT embeddings
   - Учитывает смысл, а не только точное совпадение слов
   - Диапазон: 0-1, выше = лучше

### Тестовые вопросы:
Вопросы составлены на основе реальных мероприятий в Алматы с sxodim.com.

In [ ]:
# Тестовые вопросы по мероприятиям Алматы
test_questions = [
    # Общие вопросы
    "Куда можно сходить в Алматы на выходных?",
    "Что интересного посмотреть в Алматы сегодня?",
    "Посоветуй развлечения в Алматы",
    
    # Стендап и юмор
    "Где в Алматы можно посмотреть стендап?",
    "Какие комедийные шоу есть в Алматы?",
    "Сколько стоят билеты на стендап в Punch?",
    
    # Концерты
    "Какие концерты будут в Алматы?",
    "Где послушать джаз в Алматы?",
    "Что такое EverJazz?",
    
    # Кино
    "Есть ли необычные кинотеатры в Алматы?",
    "Что такое киноужин от VkusKino?",
    
    # Квизы и развлечения
    "Где поиграть в квиз в Алматы?",
    "Что такое Квиз, плиз?",
    
    # Театр и спектакли
    "Какие спектакли идут в Алматы?",
    "Куда сходить с детьми в Алматы?",
]

# Эталонные ответы
reference_answers = [
    "В Алматы на выходных можно посетить стендап-шоу в Punch Stand Up Club, джазовые концерты в EverJazz, кинопоказы под открытым небом в Open Cinema или поиграть в Квиз, плиз!",
    "Сегодня в Алматы можно посетить стендап-концерты, джазовые выступления в клубе EverJazz, кинопоказы или интеллектуальные игры. Рекомендую проверить афишу на sxodim.com.",
    "В Алматы много развлечений: стендап в Punch Stand Up Club (от 3000 тенге), джаз в EverJazz, квизы в барах, киноужины от VkusKino, спектакли и концерты.",
    "Стендап в Алматы можно посмотреть в Punch Stand Up Club по адресу ул. Каирбекова, 35А. Там регулярно проходят шоу «4Коня», «Само собой» и другие комедийные программы.",
    "В Алматы есть несколько комедийных площадок: Punch Stand Up Club с шоу «4Коня» и импровизациями, Stand Up Camp с жёстким стендапом в разных барах города.",
    "Билеты на стендап в Punch Stand Up Club стоят от 2000-3000 тенге. Шоу проходят несколько раз в неделю, начало обычно в 19:30 или 21:30.",
    "В Алматы проходят концерты разных жанров: джаз в EverJazz, электронная музыка, рок-концерты в Motor Club, а также выступления российских и казахстанских артистов.",
    "Джаз в Алматы можно послушать в клубе EverJazz по адресу ул. Гоголя, 40Б. Там выступают местные джазмены, проходят jam session по понедельникам. Билеты от 2000 тенге.",
    "EverJazz — это джаз-клуб в Алматы на ул. Гоголя, 40Б. Здесь проходят живые концерты, джем-сейшны и выступления казахстанских джазовых музыкантов.",
    "Да, в Алматы есть Open Cinema — кинотеатр на крыше с прозрачным куполом на ул. Жандосова 58/1. Там показывают фильмы с видом на город и горы, есть пледы и бар.",
    "Киноужин от VkusKino — это уникальный формат, где во время просмотра фильма подают тематические блюда и коктейли. Например, на «Шрек 2» подают волшебные зелья и блюда из сказки.",
    "Квизы в Алматы проходят в разных барах. Самый популярный — Квиз, плиз! в Guinness pub на ул. Достык, 71. Билеты от 3000 тенге с человека.",
    "Квиз, плиз! — это интеллектуальная игра, где команды из 2-9 человек отвечают на вопросы. 7 раундов, 39 вопросов, около 2 часов. Играют более 200 тысяч команд по всему миру.",
    "В Алматы идут разные спектакли: «Сиротливый запад», «Мечта», «История двух рыцарей», музыкальный спектакль «Состояние Танго» и другие постановки.",
    "С детьми в Алматы можно сходить на интерактивный цирк Шапито, детские программы в Oi-Qaragai (лыжи, юрты), мультфильмы в Open Cinema или киноужины с семейными фильмами.",
]

print(f"📋 Подготовлено тестовых вопросов: {len(test_questions)}")

In [ ]:
# Переключаем модель в режим инференса
FastLanguageModel.for_inference(model)

# Системный промпт
SYSTEM_PROMPT = "Ты — дружелюбный помощник по мероприятиям в Алматы. Помогаешь пользователям найти интересные события, концерты, спектакли, выставки и развлечения в городе. Отвечай информативно, указывая даты, время, место и цены."

def generate_answer(question: str, max_tokens: int = 200) -> str:
    """Генерация ответа на вопрос."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]
    
    # Применяем chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Токенизация
    inputs = tokenizer([text], return_tensors="pt").to("cuda")
    
    # Генерация
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Декодирование
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Извлекаем только ответ ассистента
    if "assistant" in full_response.lower():
        parts = full_response.split("assistant")
        answer = parts[-1].strip()
    else:
        answer = full_response
    
    return answer

print("✅ Функция генерации готова")

In [ ]:
# Генерируем ответы на все тестовые вопросы
print("🔄 Генерация ответов...\n")

generated_answers = []

for i, question in enumerate(tqdm(test_questions, desc="Generating")):
    answer = generate_answer(question)
    generated_answers.append(answer)
    
    # Показываем первые 5 примеров
    if i < 5:
        print(f"\n{'='*60}")
        print(f"❓ Q{i+1}: {question}")
        print(f"🤖 Generated: {answer[:200]}..." if len(answer) > 200 else f"🤖 Generated: {answer}")
        print(f"📚 Reference: {reference_answers[i][:200]}..." if len(reference_answers[i]) > 200 else f"📚 Reference: {reference_answers[i]}")

print(f"\n✅ Сгенерировано ответов: {len(generated_answers)}")

In [ ]:
import evaluate

# Загружаем метрики
bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")
bertscore_metric = evaluate.load("bertscore")

print("✅ Метрики загружены")

In [ ]:
# 1. BLEU Score
print("📊 Вычисление BLEU...")
references_bleu = [[ref] for ref in reference_answers]  # BLEU требует список референсов

bleu_result = bleu_metric.compute(
    predictions=generated_answers,
    references=references_bleu
)

print(f"   BLEU Score: {bleu_result['score']:.2f}")

In [ ]:
# 2. ROUGE Scores
print("\n📊 Вычисление ROUGE...")

rouge_result = rouge_metric.compute(
    predictions=generated_answers,
    references=reference_answers
)

print(f"   ROUGE-1: {rouge_result['rouge1']:.4f}")
print(f"   ROUGE-2: {rouge_result['rouge2']:.4f}")
print(f"   ROUGE-L: {rouge_result['rougeL']:.4f}")
print(f"   ROUGE-Lsum: {rouge_result['rougeLsum']:.4f}")

In [ ]:
# 3. BERTScore (семантическое сходство)
print("\n📊 Вычисление BERTScore (может занять время)...")

bertscore_result = bertscore_metric.compute(
    predictions=generated_answers,
    references=reference_answers,
    lang="ru",  # Русский язык
    model_type="bert-base-multilingual-cased"  # Мультиязычная модель
)

# Средние значения
precision = sum(bertscore_result['precision']) / len(bertscore_result['precision'])
recall = sum(bertscore_result['recall']) / len(bertscore_result['recall'])
f1 = sum(bertscore_result['f1']) / len(bertscore_result['f1'])

print(f"   BERTScore Precision: {precision:.4f}")
print(f"   BERTScore Recall: {recall:.4f}")
print(f"   BERTScore F1: {f1:.4f}")

---

## Заключение

В этом ноутбуке мы:

1. ✅ Загрузили Qwen2.5-3B с 4-bit квантизацией (QLoRA)
2. ✅ Настроили LoRA адаптеры (r=32, alpha=64)
3. ✅ Подготовили SFT датасет из 1443 Q&A пар по мероприятиям Алматы
4. ✅ Обучили модель с SFTTrainer (знание фактов)
5. ✅ Применили ORPO для дружелюбного стиля ответов
6. ✅ Оценили качество с помощью BLEU, ROUGE и BERTScore

### Пайплайн обучения:
```
База (Qwen2.5-3B) → SFT (факты о мероприятиях) → ORPO (дружелюбный стиль)
```

### Следующие шаги:
- Регулярно обновлять датасет новыми мероприятиями
- Добавить RAG для актуальной информации о событиях
- Интегрировать с Telegram-ботом для пользователей
- Загрузить на HuggingFace Hub для sharing

## 8. Интерактивное тестирование

Протестируйте модель с собственными вопросами о мероприятиях в Алматы!

In [ ]:
# Интерактивный тест
def chat(question: str):
    """Простой интерфейс для тестирования модели."""
    print(f"\n❓ Вопрос: {question}")
    answer = generate_answer(question, max_tokens=300)
    print(f"\n🤖 Ответ: {answer}")
    print("-" * 60)

# Примеры использования
chat("Куда сходить в Алматы на выходных?")
chat("Где послушать живую музыку в Алматы?")
chat("Какие стендап-шоу есть в Punch Stand Up Club?")

## 9. Экспорт модели (опционально)

Можно объединить LoRA адаптеры с базовой моделью или экспортировать в GGUF для llama.cpp.

In [ ]:
# Опция 1: Сохранить merged модель в 16-bit (для HuggingFace Hub)
# model.save_pretrained_merged("shodim_almaty_merged_16bit", tokenizer, save_method="merged_16bit")

# Опция 2: Экспорт в GGUF для llama.cpp (локальный inference)
# model.save_pretrained_gguf("shodim_almaty_gguf", tokenizer, quantization_method="q4_k_m")

print("💡 Раскомментируйте нужную опцию для экспорта")

---

## Заключение

В этом ноутбуке мы:

1. ✅ Загрузили Qwen2.5-3B с 4-bit квантизацией (QLoRA)
2. ✅ Настроили LoRA адаптеры (r=32, alpha=64)
3. ✅ Подготовили датасет из 1443 Q&A пар по мероприятиям Алматы (sxodim.com)
4. ✅ Обучили модель с SFTTrainer
5. ✅ Оценили качество с помощью BLEU, ROUGE и BERTScore

### Следующие шаги:
- Регулярно обновлять датасет новыми мероприятиями
- Добавить RAG для актуальной информации о событиях
- Интегрировать с Telegram-ботом для пользователей
- Загрузить на HuggingFace Hub для sharing